In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import Counter

In [2]:
import gensim
import gensim.corpora as corpora

In [2]:
dfevents = pd.read_csv('../../data/ecommerce/events.csv')

In [3]:
dfevents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
timestamp        int64
visitorid        int64
event            object
itemid           int64
transactionid    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [4]:
def time_format(df):
    """formats the timestamp column inplace"""
    times=[]  # convert from unix time format
    for i in df['timestamp']:
        times.append(datetime.datetime.fromtimestamp(i//1000.0))
    df['timestamp']=times
time_format(dfevents)
dfevents.head()

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-01 23:02:12,257597,view,355908,NaN
1,2015-06-01 23:50:14,992329,view,248676,NaN
2,2015-06-01 23:13:19,111016,view,318965,NaN
3,2015-06-01 23:12:35,483717,view,253185,NaN
4,2015-06-01 23:02:17,951259,view,367447,NaN


In [ ]:
# dflog = dfevents.groupby('visitorid').agg('describe') # didnt finish in 30min
scrap = dfevents.loc[dfevents['event'] == 'transaction', 
                     ['visitorid', 'event']].groupby('visitorid').agg('count')
scrap.head()
# scrap.shape # number of users who made purchase 11719

In [35]:
def event_counts(df):
    """ input events df
    returns 3 DF for view_count, add_count, purchase_count
    """
    col_map = [('view', 'view_count'), ('addtocart', 'add_count'), 
               ('transaction', 'purchase_count')]
    dft = [0, 0, 0]
    for idx, tup in enumerate(col_map):
        dft[idx] = df.loc[df['event'] == tup[0], ['visitorid', 
                                'event']].groupby('visitorid').agg('count')
    return dft[0], dft[1], dft[2]

In [5]:
dfcluster = pd.read_pickle('../../data/ecommerce/cust_prod_clust2.pkl', compression='zip')
dfcluster = dfcluster[dfcluster['item_count'] < 400]
# gensim_processed = dfcluster['product_list']
seriesldamask = pd.read_pickle('../../data/ecommerce/seriesldamask.pkl', compression='zip')

In [6]:
print(dfcluster.shape)
print(seriesldamask.shape)

(405924, 2)
(405924,)


In [7]:
dfcluster = dfcluster[seriesldamask.values]
print(dfcluster.shape)
dfcluster.head()

(380127, 2)


,product_list,item_count
visitorid,,
0,"[285930, 357564, 67045]",3
2,"[216305, 325215, 342816, 325215, 342816, 25988...",8
6,"[344723, 344723, 65273, 253615, 344723, 344723]",6
7,"[164941, 139394, 226353]",3
13,"[320589, 278689]",2


In [8]:
dfcluster.index.values

array([      0,       2,       6, ..., 1407567, 1407572, 1407573])

In [9]:
clusterlist = list(dfcluster.index.values)
type(clusterlist)
clusterlist[:5]

[0, 2, 6, 7, 13]

In [10]:
clustermask = dfevents['visitorid'].isin(clusterlist)
clustermask.shape

(2756101,)

In [11]:
dfevents = dfevents.loc[clustermask]
dfevents.shape

(1587292, 5)

In [ ]:
dfview, dfadd, dfpurchase = event_counts(dfevents)
dfpurchase.head()

In [70]:
viewmask = dfevents['visitorid'].isin(dfview.index.values)
addmask = dfevents['visitorid'].isin(dfadd.index.values)
purchasemask = dfevents['visitorid'].isin(dfpurchase.index.values)
print(Counter(viewmask), Counter(addmask), Counter(purchasemask))

Counter({True: 1585025, False: 2267}) Counter({False: 1272457, True: 314835}) Counter({False: 1443235, True: 144057})


In [71]:
print(dfview.shape, dfadd.shape, dfpurchase.shape)

(379347, 1) (34681, 1) (11449, 1)


In [102]:
def set_counts(df):
    """input dfevents to set: view_count, add_count, purchase_count
    """
    df['view_count'] = 0
    df['add_count'] = 0
    df['purchase_count'] = 0
    dfview, dfadd, dfpurchase = event_counts(df)
#     viewmask = df['visitorid'].isin(dfview.index.values)
#     addmask = df['visitorid'].isin(dfadd.index.values)
#     purchasemask = df['visitorid'].isin(dfpurchase.index.values)
    for visid in df.visitorid.values:
        if visid in dfview.index.values:
            df.loc[[df['visitorid'] == visid], ['view_count']] = dfview.loc[visid]
#     for visid in df[['visitorid']].values:
        if visid in dfadd.index.values:
            df.loc[[df['visitorid'] == visid], ['add_count']] = dfadd.loc[visid]
#     for visid in df[['visitorid']].values:
        if visid in dfpurchase.index.values:
            df.loc[[df['visitorid'] == visid], ['purchase_count']] = dfpurchase.loc[visid]

In [ ]:
# set_counts(dfevents)  BROKEN
# dfevents.head(12)

In [93]:
dfview.loc[992329]

event    30
Name: 992329, dtype: int64

In [12]:
print(np.unique(dfevents['visitorid'].to_numpy())[:5])
np.unique(dfevents['visitorid'].to_numpy()).shape

[ 0  2  6  7 13]


(380127,)

In [119]:
def another_counter(df):  # BROKEN DOESN'T RUN
    """input dfevents to set: view_count, add_count, purchase_count
    maybe this function will work
    """
    uniq_id = np.unique(df['visitorid'].to_numpy())
    for visid in uniq_id:
        id_mask = df['visitorid'] == visid
        dfslice = df.loc[id_mask]
        view = sum(dfslice['event'] == 'view')
        add = sum(dfslice['event'] == 'addtocart')
        purchase = sum(dfslice['event'] == 'transaction')
        df.loc[id_mask, ['view_count']] = view
        df.loc[id_mask, ['add_count']] = add
        df.loc[id_mask, ['purchase_count']] = purchase

In [ ]:
# another_counter(dfevents)  # BROKEN, DOESN'T RUN IN 8+ HOURS
# dfevents.head(10)

In [62]:
df539 = dfevents.loc[dfevents['visitorid'] == 539]
df539

,timestamp,visitorid,event,itemid,transactionid
261350,2015-06-15 23:41:55,539,view,94371,NaN
263348,2015-06-15 23:37:53,539,view,94371,NaN
269532,2015-06-15 23:37:58,539,addtocart,94371,NaN
273133,2015-06-15 23:39:38,539,transaction,94371,14778.0
276254,2015-06-15 23:36:01,539,view,94371,NaN
281047,2015-06-15 23:42:04,539,addtocart,94371,NaN
309666,2015-06-16 10:09:52,539,view,94371,NaN


In [170]:
# dfevents.loc[dfevents["event"] == "addtocart"]
mask1 = dfevents['visitorid'] == 1155978
mask2 = dfevents['visitorid'] == 539
selector = mask1 | mask2
dfevents.loc[mask1 | mask2]

,timestamp,visitorid,event,itemid,transactionid
261350,2015-06-15 23:41:55,539,view,94371,NaN
263348,2015-06-15 23:37:53,539,view,94371,NaN
269532,2015-06-15 23:37:58,539,addtocart,94371,NaN
273133,2015-06-15 23:39:38,539,transaction,94371,14778.0
276254,2015-06-15 23:36:01,539,view,94371,NaN
281047,2015-06-15 23:42:04,539,addtocart,94371,NaN
309666,2015-06-16 10:09:52,539,view,94371,NaN
2319744,2015-07-13 12:48:36,1155978,view,430050,NaN
2329802,2015-07-13 12:53:36,1155978,view,430050,NaN
2332797,2015-07-13 12:53:53,1155978,view,239248,NaN


In [14]:
sum(df539['event'] == 'view')

4

In [15]:
# dfevents.loc[dfevents['visitorid'] == 539, ['view_count']] = 4
df539.sort_values(by=['timestamp'])

,timestamp,visitorid,event,itemid,transactionid
276254,2015-06-15 23:36:01,539,view,94371,NaN
263348,2015-06-15 23:37:53,539,view,94371,NaN
269532,2015-06-15 23:37:58,539,addtocart,94371,NaN
273133,2015-06-15 23:39:38,539,transaction,94371,14778.0
261350,2015-06-15 23:41:55,539,view,94371,NaN
281047,2015-06-15 23:42:04,539,addtocart,94371,NaN
309666,2015-06-16 10:09:52,539,view,94371,NaN


In [ ]:
def time_delta(df):
    """input dfevents to set: time_delta
    latest - oldest timestamp per user
    """
    uniq_id = np.unique(df['visitorid'].to_numpy())
    for visid in uniq_id:
        id_mask = df['visitorid'] == visid
        dfslice = df.loc[id_mask].sort_values(by=['timestamp'])
        deltatype = dfslice.iloc[-1, 0] - dfslice.iloc[0, 0]
        df.loc[id_mask, ['time_delta']] = deltatype
        df.loc[id_mask, ['time_hour']] = deltatype.delta / 3600000000000 # ns to minutes

In [ ]:
dfsam = dfevents.iloc[20000:40000].copy()
print(dfsam.shape)
# dfsam['time_delta'] = np.nan
# dfsam['time_hour'] = np.nan
# dfsam['view_count'] = np.nan
# dfsam['add_count'] = np.nan
# dfsam['purchase_count'] = np.nan
dfsam.head()

In [44]:
scrap = pd.DataFrame([[1, 3, 4], [1, 5, 6], [1, 8, 9], [2, 10, 11], [2, 12, 13]])

In [85]:
def test_agg(row):
    c0 = np.max(row[1])
    c1 = len(row[2])
    return c0, c1

In [86]:
scrap.groupby(0).agg(test_agg)

,1,2
0,,
1,8,3
2,12,2


In [181]:
df539.sort_values(by=['timestamp'])

,timestamp,visitorid,event,itemid,transactionid
276254,2015-06-15 23:36:01,539,view,94371,NaN
263348,2015-06-15 23:37:53,539,view,94371,NaN
269532,2015-06-15 23:37:58,539,addtocart,94371,NaN
273133,2015-06-15 23:39:38,539,transaction,94371,14778.0
261350,2015-06-15 23:41:55,539,view,94371,NaN
281047,2015-06-15 23:42:04,539,addtocart,94371,NaN
309666,2015-06-16 10:09:52,539,view,94371,NaN


In [183]:
df539 = df539.assign(mintimestamp=lambda x: x.timestamp)

In [50]:
from sklearn.preprocessing import OneHotEncoder
# OneHotEncoder(categories='auto', drop=None, sparse=True, 
# dtype=<class 'numpy.float64'>, handle_unknown='error')

In [171]:
enc = OneHotEncoder(sparse=False, dtype=np.uint8)
X = df539['event'].values.reshape(-1, 1)
print(X)
enc.fit(X)
print(enc.categories_)
X = enc.transform(X)
X

[['view']
 ['view']
 ['addtocart']
 ['transaction']
 ['view']
 ['addtocart']
 ['view']]
[array(['addtocart', 'transaction', 'view'], dtype=object)]


array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]], dtype=uint8)

In [184]:
dfonehot = pd.DataFrame(X, columns=enc.categories_[0])
# dfonehot
# df539
# df539.join(dfonehot, lsuffix='_caller', rsuffix='_other')
df539onehot = pd.concat([df539.reset_index(), dfonehot], axis=1)
df539onehot

,index,timestamp,visitorid,event,itemid,transactionid,mintimestamp,addtocart,transaction,view
0,261350,2015-06-15 23:41:55,539,view,94371,NaN,2015-06-15 23:41:55,0,0,1
1,263348,2015-06-15 23:37:53,539,view,94371,NaN,2015-06-15 23:37:53,0,0,1
2,269532,2015-06-15 23:37:58,539,addtocart,94371,NaN,2015-06-15 23:37:58,1,0,0
3,273133,2015-06-15 23:39:38,539,transaction,94371,14778.0,2015-06-15 23:39:38,0,1,0
4,276254,2015-06-15 23:36:01,539,view,94371,NaN,2015-06-15 23:36:01,0,0,1
5,281047,2015-06-15 23:42:04,539,addtocart,94371,NaN,2015-06-15 23:42:04,1,0,0
6,309666,2015-06-16 10:09:52,539,view,94371,NaN,2015-06-16 10:09:52,0,0,1


In [190]:
def agg_func(row):
    """function for agging events onehot df
    columns: 8 after onehot (subtracting 1 for visitorid to index)
    """
    # c0, c1, c2, c3, c4, c5, c6, c7 = [0,0,0,0,0,0,0,0]
    c0 = row['index']
    c1 = np.max(row['timestamp'])
    c2 = row['event']
    c3 = np.unique(row['itemid']).shape[0] # number of items acted on
    c4 = np.max(row['transactionid'])
    c45 = np.min(row['mintimestamp']) # preparing for time_delta assign
    c5 = np.sum(row['addtocart'])
    c6 = np.sum(row['transaction'])
    c7 = np.sum(row['view'])
    return c0, c1, c2, c3, c4, c45, c5, c6, c7

In [189]:
df539onehot

,index,timestamp,visitorid,event,itemid,transactionid,mintimestamp,addtocart,transaction,view
0,261350,2015-06-15 23:41:55,539,view,94371,NaN,2015-06-15 23:41:55,0,0,1
1,263348,2015-06-15 23:37:53,539,view,94371,NaN,2015-06-15 23:37:53,0,0,1
2,269532,2015-06-15 23:37:58,539,addtocart,94371,NaN,2015-06-15 23:37:58,1,0,0
3,273133,2015-06-15 23:39:38,539,transaction,94371,14778.0,2015-06-15 23:39:38,0,1,0
4,276254,2015-06-15 23:36:01,539,view,94371,NaN,2015-06-15 23:36:01,0,0,1
5,281047,2015-06-15 23:42:04,539,addtocart,94371,NaN,2015-06-15 23:42:04,1,0,0
6,309666,2015-06-16 10:09:52,539,view,94371,NaN,2015-06-16 10:09:52,0,0,1


In [191]:
# df539onehot.groupby('visitorid').agg(agg_func)['event'] # this returns event Series
df539agg = df539onehot.groupby('visitorid').agg(agg_func)
df539agg

,index,timestamp,event,itemid,transactionid,mintimestamp,addtocart,transaction,view
visitorid,,,,,,,,,
539,0 261350 1 263348 2 269532 3 27313...,2015-06-16 10:09:52,0 view 1 view 2 addto...,1,14778.0,2015-06-15 23:36:01,2,1,4


In [120]:
np.max(df539agg.timestamp.iloc[0]) - np.min(df539agg.timestamp.iloc[0])

Timedelta('0 days 10:33:51')

In [235]:
# ADD TIME_DELTA COLUMN AND TIME_HOUR COLUMN
# df539agg = df539agg.assign(time_delta=lambda x: x.timestamp - x.mintimestamp)
# df539agg
# df = df.assign(time_hour=lambda x: x.time_delta.iloc[0].delta / 3600000000000) OLDDDD
# df539agg.assign(time_hour=lambda x: x.time_delta.delta / 3600000000000)
# 'Series' object has no attribute 'delta'
# following line works!!!!!!
# df539agg = df539agg.assign(time_hour=lambda x: x.time_delta / np.timedelta64(1, 'h'))
# df539agg.time_delta / 3600000000000
# df539agg.loc[539, ['timestamp']] / np.timedelta64(1, 'h')
# df539agg.iloc[0, -1]
df539agg

,index,timestamp,event,product_count,transactionid,mintimestamp,addtocart,transaction,view,time_delta,time_hour
visitorid,,,,,,,,,,,
539,0 261350 1 263348 2 269532 3 27313...,2015-06-16 10:09:52,0 view 1 view 2 addto...,1,14778.0,2015-06-15 23:36:01,2,1,4,10:33:51,10.564167


In [192]:
df539agg.rename(columns={"itemid":"product_count"}, inplace=True)

In [145]:
# now make the made_purchase column
# df539agg = df539agg.assign(made_purchase=lambda x: np.uint8(x.transaction > 0))

In [ ]:
df539agg

In [240]:
# try loading the pickle from the script
dftoyscript = pd.read_pickle('../../data/ecommerce/dftoy_script.pkl', compression='zip')
dftoyscript

,index,timestamp,event,product_count,transactionid,mintimestamp,addtocart,transaction,view,time_delta,time_hour,made_purchase
visitorid,,,,,,,,,,,,
539,0 261350 1 263348 2 269532 3 27313...,2015-06-16 10:09:52,0 view 1 view 2 addto...,1,14778.0,2015-06-15 23:36:01,2,1,4,0 days 10:33:51,10.564167,1
1155978,7 2319744 8 2329802 9 2332797 10 ...,2015-07-31 18:20:36,7 view 8 view 9 ...,3,4316.0,2015-07-13 12:48:36,1,1,8,18 days 05:32:00,437.533333,1


In [ ]:
# time delta doesn't work
# might have fixed it
dfevents.loc[mask1 | mask2]

In [238]:
# drop columns test
droplist = ['index', 'event', 'mintimestamp']
dftoyscript.drop(columns=droplist, inplace=True)

In [ ]:
dfreal = pd.read_pickle('../../data/ecommerce/dfmodel_script.pkl', compression='zip')

In [244]:
dfreal.iloc[500:580]

,timestamp,product_count,transactionid,addtocart,transaction,view,time_delta,time_hour,made_purchase
visitorid,,,,,,,,,
1972,2015-06-30 22:15:54,5,NaN,0,0,7,16 days 21:30:43,405.511944,0
1973,2015-07-21 10:03:02,2,NaN,0,0,2,0 days 00:25:02,0.417222,0
1974,2015-08-26 00:46:42,4,NaN,0,0,4,0 days 00:03:59,0.066389,0
1975,2015-07-28 16:03:23,3,NaN,0,0,7,0 days 00:12:05,0.201389,0
1985,2015-05-14 22:25:09,3,NaN,0,0,4,0 days 00:03:07,0.051944,0
1986,2015-09-17 16:11:00,2,NaN,0,0,2,0 days 00:28:12,0.470000,0
1993,2015-06-25 11:51:33,2,NaN,0,0,2,0 days 00:01:01,0.016944,0
1997,2015-06-28 09:50:24,2,NaN,0,0,2,0 days 00:00:19,0.005278,0
1999,2015-07-08 16:27:11,3,NaN,0,0,3,0 days 00:09:37,0.160278,0
